## RNN vainilla

Para una RNN simple como se muestra en la figura:

![vanilla rnn](https://github.com/mcd-unison/pln/blob/main/labs/RNN/vanilla_rnn.PNG?raw=1)


La activación de los estados ocultos están dados por:      

$h^{<t>}=g(W_{hh}h^{<t-1>} + W_{hx}x^{<t>} + b_h)$                                        


este ejemplo lo vams a hacer usando exclusivamente `numpy` para entender el modelo.

In [1]:
import numpy as np
from time import perf_counter

Vamos entonces a desarrollar la función de alimentación a adelante de una RNN

In [2]:
def sigmoid(x):
    # Calcula la función logística

    ## INICIA CODIGO
    return 1/(1+np.exp(-x))
    ## ACABA CODIGO

In [3]:
def forward_V_RNN(inputs, weights):
    # Forward propagation para una RNN vanilla
    x_t, h_t_prev = inputs

    # weights.
    w_hh, w_xh, b_h = weights

    ### INICIA CODIGO ###
    # Nuevo estado oculto

    # Operación lineal
    z_t = w_hh @ h_t_prev + w_xh @ x_t + b_h

    # Activación
    h_t = sigmoid(z_t)

    ### ACABA CODIGO ###

    return h_t

Vamos a probar como funciona

In [10]:
# Data

nh = 2   # Dimensión del vector de variables ocultas
nx = 3   # Dimensión del vector de entrada



# Si prefieres valores aleatorios, descomenta lo siguiente:

w_hh = np.random.standard_normal((nh,nh))
w_hx = np.random.standard_normal((nh,nx))
h_t_prev = np.random.standard_normal((nh,1))
x_t = np.random.standard_normal((nx,1))

# Aplicando un solo paso
h_t = forward_V_RNN([x_t, h_t_prev], [w_hh, w_hx, b_h])

print("\nValor h_t:")
print(h_t, "\n")




Valor h_t:
[[0.30795419]
 [0.17723575]] 



## RNN tipo LSTM

Una LST es un modelo como el que se muestra en la figura, con todo y sus ecuaciones

![](https://github.com/mcd-unison/pln/blob/main/labs/RNN/LSTM.jpg?raw=1)

Como podemos ver tenemos 3 vectores de entrada a la celda:

- $h^{<t-1>}$ el vector de variables ocultas provenientes de un paso anterior,
- $C^{<t-1>}$ el vector de valores de celda (memoria de largo plazo) provenientes de un paso anterior,
- $x^{<t>}$ el vector de variables de entrada. Idealmente debería estar normalizado entre -1 y 1 cada uno de los valores de entrada.

Como podemos ver tenemos varias operaciones:

- Una compuerta de olvido $f$ que depende de $h^{<t-1>}$ y $x^{<t>}$ cuya salida es un vector del tamaño de las variables ocultas con valores entre 0 y 1 con la importancia que debe tener el valor de celda anterior (memoria de largo plazo)

- Una compuerta de entrada $i$ que depende de $h^{<t-1>}$ y $x^{<t>}$ cuya salida es un vector del tamaño de las variables ocultas con valores entre 0 y 1 con la importancia que debe tener la activación de la celda actual (memoria de corto plazo)

- Una compuerta de salida $i$ que depende de $h^{<t-1>}$ y $x^{<t>}$ cuya salida es un vector del tamaño de las variables ocultas con valores entre 0 y 1 con la importancia que debe tener el valor de celda actual en el valor de la de la variable oculta correspondiente.

- El calculo de la activación actual, que depende de $h^{<t-1>}$ y $x^{<t>}$, el cual se hace con una tangente hiperbólica, para mantener los valores entre -1 y 1.


Hagamos entonces una celda LSTM


In [11]:
def forward_LSTM(inputs, weights):
    # Forward propagation para una RNN tipo LSTM
    x_t, h_t_prev, C_t_prev = inputs

    # weights.
    Ui, Wi, Uf, Wf, Uo, Wo, U, W = weights

    ### INICIA CODIGO ###
    # Nuevo estado oculto y valor de celda

    # Compuerta de entrada
    i = sigmoid(x_t @ Ui + h_t_prev @ Wi)

    # Compuerta de olvido
    f = sigmoid(x_t @ Uf + h_t_prev @ Wf)

    # Compuerta de salida
    o = sigmoid(x_t @ Uo + h_t_prev @ Wo)

    # Valor de celda de memoria de corto plazo
    C_t_short = np.tanh(x_t @ U + h_t_prev @ W)

    # Valor de celda de memoria de corto y largo plazo
    C_t = sigmoid(f * C_t_prev + i * C_t_short)

    # Valor de variable oculta
    h_t =np.tanh(C_t) * o

    ### END CODE HERE ###

    return h_t, C_t

Vamos a probar como funciona

In [15]:
# Data

nh = 2   # Dimensión del vector de variables ocultas
nx = 3   # Dimensión del vector de entrada

Ui = np.random.standard_normal((nx,nh))
Wi = np.random.standard_normal((nh,nh))

Uf = np.random.standard_normal((nx,nh))
Wf = np.random.standard_normal((nh,nh))

Uo = np.random.standard_normal((nx,nh))
Wo = np.random.standard_normal((nh,nh))

U = np.random.standard_normal((nx,nh))
W = np.random.standard_normal((nh,nh))


h_t_prev = 2 * np.random.standard_normal((1,nh)) - 1
C_t_prev = np.random.standard_normal((nh,1))
x_t = 2 * np.random.standard_normal((1,nx)) - 1

# Aplicando un solo paso
h_t, C_t = forward_LSTM(
    [x_t, h_t_prev, C_t_prev],
    [Ui, Wi, Uf, Wf, Uo, Wo, U, W]
)

print("\nValor h_t:")
print(h_t, "\n")

print("\nValor C_t:")
print(C_t, "\n")



Valor h_t:
[[0.03303468 0.03759646]
 [0.039262   0.03763813]] 


Valor C_t:
[[0.34081786 0.34032435]
 [0.41189118 0.34073141]] 



## La función `scan`para el cálculo de BPTT

La función `scan` se usa para calcular la propagación hacia adelante. Si la funcións e implementa en un *framework* como *Tensorflow* o *pytorch*, entonces se puede ir guardando los gradientes de cada aplicación a lo largo del tiempo y usarlos en el calculo del gradiente para la función de aprendizaje.

Aquí solo vamos a tratar de mostrar como funcionaría dicha función, la cual recibe:

- `elems` : lista de entradas (`X`)
- `weights` : los parámetros que necesita la función de feedforward para su cálculo (pesos)
- `h_0` : estado oculto inicial

`scan` va por todos los valores de `x` en `elems`, llama la función de feedforward con los argumentos necesarios, guarda el estado oculto `h_t` y agrega el valor de `h_t` a una lista.

Vamos a hacer la función de scan para una celda tipo RNN vainilla

In [16]:
def scan_V_RNN(elems, weights, h_0=None): # Forward propagation for RNNs
    h_t = h_0
    h = []
    for x in elems:
        h_t = forward_V_RNN([x, h_t], weights)
        h.append(h_t)
    return h, h_t

Vamos a probar inicializando una posible red RNN vainilla en un probable pornblema de PLN

In [17]:
np.random.seed(10)

emb = 128                       # Embedding
T = 256                         # Tamaño de secuencia de tokens
h_dim = 16                      # Estados ocultos

h_0 = np.zeros((h_dim, 1))      # Estado inicial

# Inicialización aleatoria de pesos y sesgos
Whh = np.random.standard_normal((h_dim, h_dim))
Wxh = np.random.standard_normal((h_dim, emb))
bh = np.random.standard_normal((h_dim, 1))

# Inicialización aleatoria de una secuencia de tokens (en embeddings)
X = np.random.standard_normal((T, emb, 1))

weights = [Whh, Wxh, bh]

In [19]:
# vanilla RNNs
tic = perf_counter()
h, h_T = scan_V_RNN(X, weights, h_0)
toc = perf_counter()
RNN_time=(toc-tic)*1000
print (f"Tomó {RNN_time:.2f}ms ejecutar el método de RNN vainilla.")


Tomó 6.36ms ejecutar el método de RNN vainilla.


**Desarrolla la función de scan para LSTM y prueba con la misma secuencia de entradas para una red LSTM**

In [20]:
# Función scan para LSTM

# INICIA CODIGO
def scan_LSTM(elems, weights, h0, C0):
  ht = h0
  Ct = C0
  h = []
  for x in elems:
    ht, Ct = forward_LSTM([x, ht, Ct], weights)
    h.append(ht)
  return h, ht
# TERMINA CODIGO

In [21]:
# Inicialización de variables

# INICIA CODIGO

np.random.seed(10)

emb = 128                       # Embedding
T = 256                         # Tamaño de secuencia de tokens
h_dim = 16                      # Estados ocultos

# Inicialización aleatoria de pesos y sesgos
h_0 = np.zeros((1, h_dim))      # Estado inicial
C_0 = np.zeros((h_dim, 1))
Ui = np.random.randn(emb, h_dim)
Wi = np.random.randn(h_dim, h_dim)
Uf = np.random.randn(emb, h_dim)
Wf = np.random.randn(h_dim, h_dim)
Uo = np.random.randn(emb, h_dim)
Wo = np.random.randn(h_dim, h_dim)
U = np.random.randn(emb, h_dim)
W = np.random.randn(h_dim, h_dim)

# Inicialización aleatoria de una secuencia de tokens (en embeddings)
X = np.random.standard_normal((T, emb))

weights = [Ui, Wi, Uf, Wf, Uo, Wo, U, W]
# TERMINA CODIGO

In [24]:
# Probando la función de scan

# INICIA CODIGO
tic = perf_counter()
h, h_T = scan_LSTM(X, weights, h_0, C_0)
toc = perf_counter()
LSTM_time=(toc-tic)*1000
print (f"Tomó {LSTM_time:.2f}ms ejecutar el método de RNN vainilla.")
# TERMINA CODIGO

Tomó 36.87ms ejecutar el método de RNN vainilla.
